# TP 02 - Positional Encoding et Bases de l'Attention

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Comprendre le Positional Encoding et les bases de l'attention

---

## Objectifs pédagogiques

À la fin de cette session, vous serez capable de :
1. Expliquer pourquoi le **Positional Encoding** est nécessaire
2. Calculer le PE avec la formule sin/cos
3. Comprendre la relation entre **similarité** et **produit scalaire**
4. Calculer les **scores d'attention** étape par étape

---

**Note** : Ce TP pose les fondations. Le TP suivant implémentera l'attention complète.

## 0. Installation et imports

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy transformers -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

# Configuration
torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")
print(f"GPU disponible: {torch.cuda.is_available()}")

---

## 1. Rappel du TP1 - Chargement CamemBERT

In [ ]:
# Récuperation d'un embedding sur un modèle déjà entrainé (BERT)
from transformers import CamembertTokenizer, CamembertModel

print("Chargement de CamemBERT (modèle français)...")
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model_camembert = CamembertModel.from_pretrained("camembert-base")

# On utilise UNIQUEMENT la couche d'embeddings
embedding_layer = model_camembert.embeddings.word_embeddings

print("✅ CamemBERT chargé !")
print(f"   Dimension des embeddings : {embedding_layer.embedding_dim}")

def get_french_embeddings(phrase, target_dim=100):
    """
    Extrait les embeddings d'une phrase française.
    """
    inputs = tokenizer(
        phrase,
        is_split_into_words=True,
        padding=True,
        truncation=True,
        return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    with torch.no_grad():
        embeddings_768 = embedding_layer(inputs["input_ids"][0])

    if not hasattr(get_french_embeddings, 'projection'):
        torch.manual_seed(42)
        get_french_embeddings.projection = torch.randn(768, target_dim) / 30

    embeddings = embeddings_768 @ get_french_embeddings.projection
    return embeddings, tokens

In [ ]:
# Exercice de rappel : similarité
mots = ["Paris", "tour", "Eiffel"]
embeddings, tokens = get_french_embeddings(mots)

emb_paris = embeddings[tokens.index("▁Paris")]
emb_tour = embeddings[tokens.index("▁tour")]
emb_eiffel = embeddings[tokens.index("▁Eiffel")]

emb_tour_eiffel = (emb_tour + emb_eiffel) / 2

sim_paris_tour_eiffel = F.cosine_similarity(
    emb_paris.unsqueeze(0),
    emb_tour_eiffel.unsqueeze(0)
)

sim_paris_tour = F.cosine_similarity(
    emb_paris.unsqueeze(0),
    emb_tour.unsqueeze(0)
)

print(f"Similarité Paris / tour Eiffel : {sim_paris_tour_eiffel.item():.4f}")
print(f"Similarité Paris / tour : {sim_paris_tour.item():.4f}")
print("\n→ Paris est plus proche de 'tour Eiffel' que de 'tour' seul")

---

## 2. Introduction : Pourquoi l'attention ?

Jusqu'en 2017, la majorité des modèles de langage (RNN & LSTM) lisaient les phrases séquentiellement, de gauche à droite. Cette approche présente deux limites majeures :

- elle est **lente**,
- un mot placé au début de la phrase a du mal à intégrer des informations situées beaucoup plus loin.

Le **Transformer** rompt complètement avec cette logique.

Dans un Transformer, tous les mots de la phrase sont traités **simultanément**.

Prenons la phrase :

*"Le chat dort sur le canapé"*

Elle contient six mots : le modèle reçoit donc six éléments en parallèle.

La première étape consiste à transformer chaque mot en nombres : ce sont les **embeddings**.

Chaque mot est représenté par un vecteur de dimension fixe (par exemple 100 ou 512), qui encode des informations sémantiques.

À ce stade :
- on ne manipule plus du texte,
- mais une suite de vecteurs numériques.

Cependant, ces vecteurs sont encore indépendants les uns des autres. Le vecteur correspondant à "chat" ne sait rien de "dort" ou de "canapé".

👉 **L'attention va précisément servir à créer ces liens.**

### 2.1 L'architecture Transformer

Le **Transformer** (2017) abandonne la récurrence. Chaque mot peut regarder **tous les autres directement** :

```
Entrée: "Le chat dort sur le canapé" (6 tokens)
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
┌─────────────────────────────────────────────┐
│          EMBEDDINGS (6 vecteurs)            │
└─────────────────────────────────────────────┘
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
┌─────────────────────────────────────────────┐
│             SELF-ATTENTION                  │
│   Chaque vecteur regarde les 5 autres       │
│   → Enrichit chaque mot avec le CONTEXTE    │
└─────────────────────────────────────────────┘
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
       (6 vecteurs enrichis)
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
┌─────────────────────────────────────────────┐
│        FEED-FORWARD (par position)          │
│   Exploite le contexte enrichi              │
│   (comme un réseau de neurones classique)   │
└─────────────────────────────────────────────┘
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
      Sortie: 6 vecteurs transformés
```

**Points clés** :
- **Entrée = Sortie** : Si une phrase contient 6 tokens, le Transformer produit 6 vecteurs en sortie, un par token, enrichis par le contexte.
- **Taille variable** : Tu peux entrer 5, 50, ou 500 mots (jusqu'à une limite : 512 pour BERT, 128K pour GPT-4)
- **Self-Attention** : Donne du contexte à chaque mot
- **Feed-Forward** : Exploite ce contexte (transformation non-linéaire)

**Que sort le Transformer ? Quelle utilité ?**

Le Transformer produit des **vecteurs enrichis** (représentations). Une couche de sortie (ajoutée selon la tâche) les transforme en résultat :
- **Classification** → probabilité par classe (ex: 70% positif, 30% négatif)
- **Génération** → probabilité du prochain mot
- **Traduction** → phrase dans l'autre langue

### Comment les mots entrent dans le Transformer ?

Chaque mot passe par **deux étapes** avant d'entrer :

```
Mot "chat" (position 1)
        │
        ▼
┌─────────────────────────────────────────────────┐
│ Token Embedding (fixe pour chaque token)        │
│ "chat" → [0.8, 0.1, 0.3, ...]                   │
└─────────────────────────────────────────────────┘
        │
        + (addition)
        │
┌─────────────────────────────────────────────────┐
│ Positional Encoding (fixe pour chaque position) │
│ position 1 → [0.0, 0.1, 0.0, ...]               │
└─────────────────────────────────────────────────┘
        │
        ▼
Vecteur d'entrée = [0.8, 0.2, 0.3, ...]
```

**Deux composants distincts :**

| Composant | Taille | Rôle |
|-----------|--------|------|
| Token embeddings | ~50k × dim | "Qui suis-je ?" (sens du mot) |
| Positional encodings | max_len × dim | "Où suis-je ?" (position dans la phrase) |

**Pourquoi c'est important ?** Sans le positional encoding, le modèle ne distinguerait pas :
- *"Le chat mange la souris"*
- *"La souris mange le chat"*

(Mêmes tokens, ordre différent → sens opposé !)

C'est l'empilement de ces blocs d'attention avec des blocs de Feed-Forward qui constitue le Transformer.

### 2.2 Ce qu'on va construire

```
    TRANSFORMER
    ┌────────────────────────────┐
    │  Embedding + Positional    │
    ├────────────────────────────┤
    │ ┌────────────────────────┐ │
    │ │   SELF-ATTENTION  ◀────┼─┼─── Sessions 2-3
    │ └────────────────────────┘ │
    │ ┌────────────────────────┐ │
    │ │     FEED-FORWARD       │ │
    │ └────────────────────────┘ │
    │         × N blocs          │
    ├────────────────────────────┤
    │     Couche de sortie       │
    └────────────────────────────┘
```

**Plan du cours** :
- **Session 1** : Fondamentaux NLP (tokenization, embeddings)
- **Session 2** : PE + bases de l'attention (ce TP)
- **Session 3** : Implémenter l'attention complète + intro Multi-Head
- **Session 4** : Multi-Head + Transformer + masque causal
- **Sessions 5-6** : Projets Mini-GPT

### 2.3 L'idée clé de l'attention

L'attention répond à la question : **"Pour comprendre ce mot, quels autres mots dois-je regarder ?"**

**Exemple** : *"Le chat qui dormait sur le canapé a sauté"*
- Pour comprendre **"a sauté"** → regarder **"chat"** (le sujet, pas "canapé")

Grâce à l'attention :
- chaque mot intègre l'information pertinente des autres mots,
- les mots importants contribuent davantage au vecteur final,
- l'ordre et le contexte sont pris en compte sans lecture séquentielle.

Le mot "chat" ne représente plus un animal abstrait, mais un chat qui dort sur un canapé.

---

### Pour approfondir RNN/LSTM (optionnel)

**Vidéos en français** :
- [Machine Learnia - Les RNN expliqués](https://www.youtube.com/watch?v=EL439RMv3Xc) (~20 min)
- [Science4All - Comprendre les LSTM](https://www.youtube.com/watch?v=WCUNPb-5EYI) (~15 min)

**Articles en français** :
- [Pensée Artificielle - Introduction aux RNN](https://www.penseeartificielle.fr/comprendre-reseaux-neurones-recurrents-rnn/)
- [DataScientest - LSTM expliqué simplement](https://datascientest.com/lstm-tout-savoir)

---

## 3. Le problème : l'ordre des mots

Les embeddings seuls ne capturent pas la **position** des mots dans la phrase.

```
"Le chat mange la souris"  ≠  "La souris mange le chat"
```

Pourtant, les mêmes mots ont les mêmes embeddings !

---

## 4. Positional Encoding

### La formule

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

- **pos** : position dans la séquence (0, 1, 2, ...)
- **i** : indice de la dimension
- **d_model** : dimension totale des embeddings

In [ ]:
def get_positional_encoding(seq_len, d_model):
    """
    Génère le positional encoding avec la formule sin/cos.
    
    Args:
        seq_len: longueur de la séquence
        d_model: dimension des embeddings
    
    Returns:
        Tensor de shape (seq_len, d_model) avec le PE
    
    Rappel des formules:
        PE(pos, 2i)   = sin(pos / 10000^(2i/d_model))
        PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
    """
    # TODO: Créer un tensor des positions (0, 1, 2, ..., seq_len-1)
    # Indice: torch.arange() puis .unsqueeze(1) pour avoir shape (seq_len, 1)
    position = None  # TODO
    
    # TODO: Calculer le terme de division 10000^(2i/d_model)
    # Indice: utiliser torch.exp et torch.arange(0, d_model, 2)
    # Note: on calcule -log(10000)/d_model pour la stabilité numérique
    div_term = None  # TODO
    
    # TODO: Créer le tensor PE de zéros avec la bonne shape
    pe = None  # TODO
    
    # TODO: Remplir les dimensions paires (0, 2, 4, ...) avec sin
    # Indice: pe[:, 0::2] sélectionne les colonnes paires
    # TODO
    
    # TODO: Remplir les dimensions impaires (1, 3, 5, ...) avec cos
    # Indice: pe[:, 1::2] sélectionne les colonnes impaires
    # TODO
    
    return pe

# Test de votre implémentation
pe_test = get_positional_encoding(4, 8)
if pe_test is not None:
    print(f"Shape du PE: {pe_test.shape}")
    print(f"PE[0, 0] (sin de pos 0): {pe_test[0, 0]:.4f}")
    print(f"PE[0, 1] (cos de pos 0): {pe_test[0, 1]:.4f}")
    print(f"PE[1, 0] (sin de pos 1): {pe_test[1, 0]:.4f}")
else:
    print("⚠️ Implémentez la fonction get_positional_encoding")

### Exercice 1 : Pourquoi le PE est nécessaire ?

In [ ]:
print("=== Exercice 1 : Pourquoi le Positional Encoding ? ===")

phrase_a = ["Pikachu", "attaque", "Dracaufeu"]
phrase_b = ["Dracaufeu", "attaque", "Pikachu"]

def get_same_embeddings(tokens):
    """Retourne embeddings identiques pour tokens identiques."""
    vocab = {"Pikachu": 0, "attaque": 1, "Dracaufeu": 2}
    torch.manual_seed(42)
    base = torch.randn(3, 100)
    return torch.stack([base[vocab[t]] for t in tokens])

emb_a = get_same_embeddings(phrase_a)
emb_b = get_same_embeddings(phrase_b)

print(f"Phrase A : {phrase_a}")
print(f"Phrase B : {phrase_b}")

# Calculer la différence
diff_sans_pe = torch.norm(emb_a - emb_b)
print(f"\nDifférence SANS PE : {diff_sans_pe:.4f}")

# Pourquoi pas identiques ?
print("\n→ La différence n'est PAS nulle car les embeddings sont dans un ordre différent.")
print("   Mais la SOMME des embeddings serait identique !")
print(f"   Somme A : {emb_a.sum():.4f}")
print(f"   Somme B : {emb_b.sum():.4f}")

### Exercice 2 : Calculer le PE manuellement

In [ ]:
print("=== Exercice 2 : Calculer le PE manuellement ===")

print("\nCalculons le PE pour d_model=4, seq_len=3")
print("\nFormule :")
print("  PE(pos, 2i)   = sin(pos / 10000^(2i/d_model))")
print("  PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))\n")

# PE(0, 0) : pos=0, i=0 (dimension paire) → sin(0 / 10000^0) = sin(0) = 0
pe_0_0 = math.sin(0 / (10000 ** (0/4)))
print(f"PE(0, 0) = sin(0 / 10000^0) = sin(0) = {pe_0_0:.4f}")

# PE(0, 1) : pos=0, i=0 (dimension impaire) → cos(0 / 10000^0) = cos(0) = 1
pe_0_1 = math.cos(0 / (10000 ** (0/4)))
print(f"PE(0, 1) = cos(0 / 10000^0) = cos(0) = {pe_0_1:.4f}")

# PE(1, 0) : pos=1, i=0 (dimension paire) → sin(1 / 10000^0) = sin(1)
pe_1_0 = math.sin(1 / (10000 ** (0/4)))
print(f"PE(1, 0) = sin(1 / 10000^0) = sin(1) = {pe_1_0:.4f}")

# Vérification avec notre fonction
pe_verif = get_positional_encoding(2, 4)
print(f"\nVérification avec get_positional_encoding :")
print(f"  PE[0, 0] = {pe_verif[0, 0]:.4f}")
print(f"  PE[0, 1] = {pe_verif[0, 1]:.4f}")
print(f"  PE[1, 0] = {pe_verif[1, 0]:.4f}")

### Exercice 3 : Propriétés du PE

In [ ]:
print("=== Exercice 3 : Vérifier les propriétés du PE ===")

seq_len_test = 10
pe_test = get_positional_encoding(seq_len_test, 100)

# Propriété 1 : Valeurs ∈ [-1, 1]
min_val = pe_test.min().item()
max_val = pe_test.max().item()
print(f"\nMin : {min_val:.4f}")
print(f"Max : {max_val:.4f}")
print(f"→ Toutes les valeurs sont bien dans [-1, 1] ✓")

# Propriété 2 : Différence entre positions consécutives
diff_pos_0_1 = torch.norm(pe_test[0] - pe_test[1]).item()
diff_pos_4_5 = torch.norm(pe_test[4] - pe_test[5]).item()
print(f"\nDistance pos 0 → 1 : {diff_pos_0_1:.4f}")
print(f"Distance pos 4 → 5 : {diff_pos_4_5:.4f}")
print(f"→ Les distances sont similaires (positions relatives)")

# Propriété 3 : Dimensions paires et impaires
dim_0_all_pos = pe_test[:, 0]  # sin de toutes les positions
dim_1_all_pos = pe_test[:, 1]  # cos de toutes les positions
print(f"\nDimension 0 (sin) pour toutes les positions :")
print(f"  {[f'{x:.3f}' for x in dim_0_all_pos.tolist()[:5]]}...")
print(f"Dimension 1 (cos) pour toutes les positions :")
print(f"  {[f'{x:.3f}' for x in dim_1_all_pos.tolist()[:5]]}...")

### Exercice 4 : Impact du PE

In [ ]:
print("=== Exercice 4 : Démonstration de l'impact du PE ===")

phrase_a = ["Le", "chat", "mange", "la", "souris"]
phrase_b = ["La", "souris", "mange", "le", "chat"]

# Vocabulaire simplifié
vocab = {"Le": 0, "chat": 1, "mange": 2, "la": 3, "souris": 4, "La": 3, "le": 0}

def get_embeddings_simple(tokens, vocab):
    torch.manual_seed(42)
    base = torch.randn(5, 100)
    return torch.stack([base[vocab[t]] for t in tokens])

emb_a = get_embeddings_simple(phrase_a, vocab)
emb_b = get_embeddings_simple(phrase_b, vocab)

# Positional encoding
pe = get_positional_encoding(5, 100)

# Embeddings + PE
emb_a_pe = emb_a + pe
emb_b_pe = emb_b + pe

# Distances SANS PE
print("\nDistances SANS Positional Encoding :")
for i, (wa, wb) in enumerate(zip(phrase_a, phrase_b)):
    dist = torch.norm(emb_a[i] - emb_b[i]).item()
    print(f"  Position {i}: '{wa}' vs '{wb}' → {dist:.4f}")

print("\nDistances AVEC Positional Encoding :")
for i, (wa, wb) in enumerate(zip(phrase_a, phrase_b)):
    dist = torch.norm(emb_a_pe[i] - emb_b_pe[i]).item()
    print(f"  Position {i}: '{wa}' vs '{wb}' → {dist:.4f}")

print("\n→ Avec PE, même les mots identiques à la même position diffèrent")
print("  car le contexte global (ordre des mots) est différent.")

### Visualisation du Positional Encoding

In [ ]:
# Visualisation du PE
pe_visu = get_positional_encoding(50, 64)

plt.figure(figsize=(12, 6))
plt.imshow(pe_visu.T, cmap='RdBu', aspect='auto')
plt.xlabel('Position dans la séquence')
plt.ylabel('Dimension')
plt.title('Positional Encoding (sin/cos)')
plt.colorbar(label='Valeur')
plt.tight_layout()
plt.show()

print("Observation :")
print("- Les basses fréquences (dimensions hautes) varient lentement")
print("- Les hautes fréquences (dimensions basses) varient rapidement")
print("- Chaque position a un pattern unique")

---

## 5. Similarité et Produit Scalaire

Avant d'aborder l'attention, comprenons le lien entre **similarité cosinus** et **produit scalaire**.

### Formules

- **Produit scalaire** : $\vec{a} \cdot \vec{b} = \sum_i a_i b_i$

- **Similarité cosinus** : $\cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{||\vec{a}|| \cdot ||\vec{b}||}$

Le produit scalaire mesure la "compatibilité" entre deux vecteurs.

In [ ]:
# Comparaison similarité cosinus vs produit scalaire
print("=" * 60)
print("SIMILARITÉ COSINUS vs PRODUIT SCALAIRE")
print("=" * 60)

v1 = torch.tensor([1.0, 2.0, 3.0])
v2 = torch.tensor([2.0, 3.0, 4.0])

dot_product = torch.dot(v1, v2)
cos_sim = F.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0))
cos_sim_manual = dot_product / (torch.norm(v1) * torch.norm(v2))

print(f"\nVecteur 1 : {v1}")
print(f"Vecteur 2 : {v2}")
print(f"\nProduit scalaire : {dot_product:.4f}")
print(f"Similarité cosinus : {cos_sim.item():.4f}")
print(f"Similarité cosinus (manuel) : {cos_sim_manual:.4f}")

print("\n✅ Le produit scalaire Q·K mesure la 'compatibilité' entre tokens")
print("   Le softmax transforme ces scores en probabilités d'attention")

---

## 6. Introduction à l'Attention

### La formule de l'attention

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

| Composant | Rôle |
|-----------|------|
| **Q** (Query) | Ce que je cherche |
| **K** (Key) | Ce que j'offre pour être trouvé |
| **V** (Value) | L'information que je transmets |

### Analogie : Bibliothèque

- **Query** = Ta question ("Je cherche un livre sur Python")
- **Key** = Les étiquettes des livres ("Python", "Java", "Cuisine"...)
- **Value** = Le contenu des livres

L'attention calcule **quels livres sont pertinents** pour ta question.

### Exercice 6.1 : Calcul des scores

In [ ]:
print("=== Exercice 6.1 : Calcul des scores ===")

# Configuration
seq_len = 3  # 3 tokens
d_k = 4      # dimension des vecteurs

# Créer Q, K, V aléatoires
torch.manual_seed(42)
Q = torch.randn(seq_len, d_k)
K = torch.randn(seq_len, d_k)
V = torch.randn(seq_len, d_k)

print("Q (Queries) - Ce que chaque token cherche :")
print(Q)
print(f"\nK (Keys) - Comment chaque token se présente :")
print(K)

# TODO: Calculer les scores d'attention : QK^T
# Chaque ligne = un token qui "interroge" tous les autres
# Indice: utilisez l'opérateur @ pour la multiplication matricielle
# K.T transpose K pour avoir shape (d_k, seq_len)
scores = None  # TODO

if scores is not None:
    print(f"\nScores (QK^T) - Compatibilité entre tokens :")
    print(scores)
    print(f"\nShape: {scores.shape}  (3 tokens × 3 tokens)")
else:
    print("\n⚠️ Calculez scores = Q @ K.T")

### Exercice 6.2 : Scaling

In [ ]:
print("=== Exercice 6.2 : Scaling ===")

# Pourquoi diviser par sqrt(d_k) ?
# → Éviter que les scores soient trop grands (gradients instables)

# TODO: Appliquer le scaling aux scores
# Divisez par la racine carrée de d_k
# Indice: math.sqrt(d_k) ou d_k ** 0.5
scaled_scores = None  # TODO

if scores is not None and scaled_scores is not None:
    print(f"Scores originaux :")
    print(scores)
    print(f"\nScores après scaling (÷√{d_k} = ÷{math.sqrt(d_k):.2f}) :")
    print(scaled_scores)
    print(f"\n→ Les valeurs sont réduites, le softmax sera plus 'doux'")
else:
    print("⚠️ Calculez d'abord les scores (exercice 6.1), puis appliquez le scaling")

### Exercice 6.3 : Softmax

In [ ]:
print("=== Exercice 6.3 : Softmax ===")

# Softmax transforme les scores en probabilités
# Chaque ligne somme à 1

# TODO: Appliquer softmax sur les scores scalés
# Indice: F.softmax(tensor, dim=-1) applique softmax sur la dernière dimension
# dim=-1 signifie qu'on normalise sur les colonnes (chaque ligne somme à 1)
attention_weights = None  # TODO

if attention_weights is not None:
    print("Poids d'attention (après softmax) :")
    print(attention_weights)

    print(f"\nVérification - Somme par ligne :")
    print(attention_weights.sum(dim=1))
    print("\n→ Chaque ligne = distribution de probabilités sur les tokens")
else:
    print("⚠️ Appliquez F.softmax() sur scaled_scores")

### Exercice 6.4 : Output

In [ ]:
print("=== Exercice 6.4 : Output ===")

# L'output = moyenne pondérée des Values par les poids d'attention

print("V (Values) - L'information de chaque token :")
print(V)

# TODO: Calculer l'output de l'attention
# C'est la multiplication matricielle des poids d'attention avec V
# Indice: output = attention_weights @ V
output = None  # TODO

if output is not None:
    print(f"\nOutput (weights @ V) :")
    print(output)
    print(f"\nShape: {output.shape}  (même que V)")

    print("\n→ Chaque token a maintenant une représentation enrichie")
    print("   qui intègre l'information des tokens 'pertinents'")
else:
    print("\n⚠️ Calculez output = attention_weights @ V")

### Visualisation de l'attention

In [ ]:
# Visualisation
tokens_demo = ["Le", "chat", "dort"]

plt.figure(figsize=(8, 6))
plt.imshow(attention_weights.detach().numpy(), cmap='Blues')
plt.xticks(range(3), tokens_demo)
plt.yticks(range(3), tokens_demo)
plt.xlabel("Tokens regardés (Keys)")
plt.ylabel("Tokens qui regardent (Queries)")
plt.title("Matrice d'attention")
plt.colorbar(label="Poids")

# Ajouter les valeurs
for i in range(3):
    for j in range(3):
        val = attention_weights[i, j].item()
        plt.text(j, i, f'{val:.2f}', ha='center', va='center',
                color='white' if val > 0.5 else 'black', fontsize=12)

plt.tight_layout()
plt.show()

---

## Récapitulatif

### Ce que nous avons appris

| Concept | Description |
|---------|-------------|
| **Positional Encoding** | Encode la position avec sin/cos, valeurs ∈ [-1, 1] |
| **Q, K, V** | Query = question, Key = étiquette, Value = contenu |
| **Scores** | QK^T = compatibilité entre tokens |
| **Scaling** | Diviser par √d_k pour stabiliser |
| **Softmax** | Transformer en probabilités |
| **Output** | Moyenne pondérée des Values |

### Formule complète

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### Prochaine session

On va :
1. Implémenter la **fonction d'attention complète**
2. Créer une **classe SelfAttention** en PyTorch
3. Visualiser l'attention sur un **vrai modèle** (DistilBERT)
4. Découvrir le **Multi-Head Attention**